## 심장마비 확률이 높은사람?
- 성별, 나이, 혈압, 콜레스테롤, 공복혈당, 최대 심박수 등의 컬럼이 있음
- 평가: ROC-AUC, 정확도(Accuracy), F1 을 구하시오
- target : output (1:심장마비 확률 높음, 0:심장마비 확률 낮음)
- csv파일 생성 : 수험번호.csv (예시 아래 참조)
~~~
id,output
41,0.633
28,0.123
222,0.355
~~~



- 1번째 풀이

In [91]:
# 1. 데이터 import
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(train.shape, test.shape)

# 2. EDA
print('=== EDA ===')
print(train.head(3))
print(test.head(3))

# print(train.info()) # -> object 없음
# print(train.describe())
# print(train.isnull().sum()) # -> null 없음
# print(train['output'].value_counts())

# 3. 전처리
print('=== 전처리 ===')
y = train.pop('output')

train = train.drop('id', axis=1)
test_id = test.pop('id')

print(train.shape, test.shape)

# 4. Feature Engnieering
print('=== Feature Engnieering ===')
# standardscaler

# 5. train/validation 분리
print('=== train/validation 분리 ===')
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, y, test_size=0.2, random_state=123)

# 6. 모델 선택, 학습, 예측
print('=== 모델 선택, 학습, 예측 ===')
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_tr, y_tr)
pred_proba = model.predict_proba(X_val)

print(pred_proba[:10,])

# 7. 평가지표 ROC-AUC
print('=== 평가지표 ===')
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_val, pred_proba[:,1]))

# 8. test 예측 결과 csv 파일 생성
print('=== test 예측 결과 ===')
pred_proba = model.predict_proba(test)
submit = pd.DataFrame({
    'id': test_id,
    'output': pred_proba[:,1]
})

print(submit.shape)
submit.to_csv('0000.csv', index=False)

ttt = pd.read_csv('0000.csv')
print(ttt.shape)
print(ttt.head(5))

(242, 15) (61, 14)
=== EDA ===
    id  age  sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  \
0   48   53    0   2     128   216    0        0       115     0      0.0   
1  124   39    0   2      94   199    0        1       179     0      0.0   
2  129   74    0   1     120   269    0        0       121     1      0.2   

   slp  caa  thall  output  
0    2    0      0       1  
1    2    0      2       1  
2    2    1      2       1  
    id  age  sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  \
0   41   48    1   1     130   245    0        0       180     0      0.2   
1   28   65    0   2     140   417    1        0       157     0      0.8   
2  222   65    1   3     138   282    1        0       174     0      1.4   

   slp  caa  thall  
0    1    0      2  
1    2    1      2  
2    1    1      2  
=== 전처리 ===
(242, 13) (61, 13)
=== Feature Engnieering ===
=== train/validation 분리 ===
=== 모델 선택, 학습, 예측 ===
[[0.94 0.06]
 [0.24 0.76]
 [0.16 0.84]
 [0

## 데이터 불러오기

In [92]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(train.shape, test.shape)

(242, 15) (61, 14)


##EDA

In [93]:
train.head(2)

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,48,53,0,2,128,216,0,0,115,0,0.0,2,0,0,1
1,124,39,0,2,94,199,0,1,179,0,0.0,2,0,2,1


In [94]:
test.head(2)

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,41,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,28,65,0,2,140,417,1,0,157,0,0.8,2,1,2


In [95]:
train['output'].value_counts()

output
1    126
0    116
Name: count, dtype: int64

In [96]:
train.isnull().sum()

id          0
age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

In [97]:
test.isnull().sum()

id          0
age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
dtype: int64

In [98]:
train.describe()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.00000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000
mean,154.706612,54.516529,0.673554,0.913223,131.070248,246.933884,0.119835,0.53719,149.590909,0.347107,1.019835,1.380165,0.785124,2.347107,0.520661
std,86.328576,9.021513,0.469885,1.020897,17.716978,52.236414,0.325441,0.53183,21.763201,0.477037,1.182137,0.621285,1.052218,0.607160,0.500608
min,0.000000,34.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.00000,88.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,81.250000,47.250000,0.000000,0.000000,120.000000,212.000000,0.000000,0.00000,133.250000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,158.500000,55.000000,1.000000,1.000000,130.000000,243.000000,0.000000,1.00000,152.000000,0.000000,0.600000,1.000000,0.000000,2.000000,1.000000
75%,228.750000,61.000000,1.000000,2.000000,140.000000,274.000000,0.000000,1.00000,165.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,302.000000,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.00000,195.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [99]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        242 non-null    int64  
 1   age       242 non-null    int64  
 2   sex       242 non-null    int64  
 3   cp        242 non-null    int64  
 4   trtbps    242 non-null    int64  
 5   chol      242 non-null    int64  
 6   fbs       242 non-null    int64  
 7   restecg   242 non-null    int64  
 8   thalachh  242 non-null    int64  
 9   exng      242 non-null    int64  
 10  oldpeak   242 non-null    float64
 11  slp       242 non-null    int64  
 12  caa       242 non-null    int64  
 13  thall     242 non-null    int64  
 14  output    242 non-null    int64  
dtypes: float64(1), int64(14)
memory usage: 28.5 KB


In [100]:
train.nunique()

id          242
age          40
sex           2
cp            4
trtbps       47
chol        139
fbs           2
restecg       3
thalachh     83
exng          2
oldpeak      38
slp           3
caa           5
thall         4
output        2
dtype: int64

## 데이터 전처리 & 피처엔지니어링

In [101]:
train = train.drop('id', axis=1)
test_id = test.pop('id')

In [102]:
test.head(3)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,65,0,2,140,417,1,0,157,0,0.8,2,1,2
2,65,1,3,138,282,1,0,174,0,1.4,1,1,2


## 검증 데이터 분리

In [103]:
from sklearn.model_selection import train_test_split

y = train.pop('output')

X_tr, X_val, y_tr, y_val = train_test_split(train, y, test_size=0.15, random_state=123)

In [104]:
X_tr.head(3)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
5,50,1,2,129,196,0,1,163,0,0.0,2,0,2
4,71,0,0,112,149,0,1,125,0,1.6,1,0,2
216,53,1,0,140,203,1,0,155,1,3.1,0,0,3


## 모델 & 평가
- 평가: ROC-AUC, 정확도(Accuracy), F1 을 구하시오

In [139]:
from sklearn.ensemble import RandomForestClassifier

# 하이퍼파라미터튜닝
# max_depth : 3 5 7
# n_estimators 트리갯수 : 100 200 400 800 1000
model = RandomForestClassifier(random_state=123, max_depth=5, n_estimators=400)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)
pred_proba = model.predict_proba(X_val)

print(pred[:10])
print(pred_proba[:5,:])

[0 1 1 1 0 0 0 0 1 1]
[[0.94 0.06]
 [0.34 0.66]
 [0.11 0.89]
 [0.41 0.59]
 [0.94 0.06]]


In [106]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
print(roc_auc_score(y_val, pred_proba[:,1]))
print(accuracy_score(y_val, pred))
print(f1_score(y_val, pred))

# baseline
# 0.9517543859649122
# 0.8918918918918919
# 0.8947368421052632

# max_depth=3
# 0.9502923976608187
# 0.918918918918919
# 0.9230769230769231

# max_depth=5
# 0.9444444444444444
# 0.8918918918918919
# 0.8947368421052632

# max_depth=7
# 0.9502923976608187
# 0.8918918918918919
# 0.8947368421052632

# max_depth=5, n_estimators=200
# 0.9473684210526316
# 0.8918918918918919
# 0.8947368421052632

# max_depth=5, n_estimators=400
# 0.9444444444444444
# 0.8918918918918919
# 0.8947368421052632


0.9444444444444444
0.8918918918918919
0.8947368421052632


In [144]:
from xgboost import XGBClassifier

# 하이퍼파라미터튜닝
# max_depth : 3(기본) 4 5 6 7 8 9 ... 12
# n_estimators 트리갯수 : 100(기본) ~ 1000 -> 올리면 learning_rate 내려야함
# learning_rate : 0.1 ~ 0.01
model = XGBClassifier(random_state=123, max_depth=7, n_estimators=600, learning_rate=0.01)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)
pred_proba = model.predict_proba(X_val)

print(pred[:10])
print(pred_proba[:5,:])

[0 1 1 0 0 0 0 0 1 0]
[[0.9659432  0.03405675]
 [0.14508873 0.85491127]
 [0.0438295  0.9561705 ]
 [0.60049224 0.3995078 ]
 [0.9768612  0.0231388 ]]


In [145]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
print(roc_auc_score(y_val, pred_proba[:,1]))
print(accuracy_score(y_val, pred))
print(f1_score(y_val, pred))

# baseline
# 0.9152046783625731
# 0.8108108108108109
# 0.8108108108108109

# max_depth=5
# 0.9239766081871346
# 0.8108108108108109
# 0.8108108108108109

# max_depth=7
# 0.9152046783625731
# 0.8108108108108109
# 0.8108108108108109

# max_depth=5, n_estimators=200
# 0.9093567251461988
# 0.8108108108108109
# 0.8108108108108109

# max_depth=5, n_estimators=400, learning_rate=0.01
# 0.9356725146198831
# 0.8648648648648649
# 0.8717948717948718

# max_depth=5, n_estimators=600, learning_rate=0.01
# 0.9298245614035088
# 0.8108108108108109
# 0.8108108108108109

0.9298245614035088
0.8108108108108109
0.8108108108108109


## 예측 및 csv 제출

In [134]:
test.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,65,0,2,140,417,1,0,157,0,0.8,2,1,2
2,65,1,3,138,282,1,0,174,0,1.4,1,1,2
3,69,1,3,160,234,1,0,131,0,0.1,1,1,2
4,50,0,1,120,244,0,1,162,0,1.1,2,0,2


In [152]:
pred_proba = model.predict_proba(test)

submit = pd.DataFrame({
    'id': test_id,
    'output': pred_proba[:, 1]

})

submit.head(5)

,id,output
0,41,0.991762
1,28,0.961777
2,222,0.533961
3,106,0.232485
4,108,0.984405


In [147]:
submit.to_csv('1111.csv', index=False)

In [148]:
pd.read_csv('1111.csv')

,id,output
0,41,0.991762
1,28,0.961777
2,222,0.533961
3,106,0.232485
4,108,0.984405
...,...,...
56,6,0.902689
57,269,0.030219
58,2,0.982365
59,14,0.963511


##정리

In [149]:
y_test = pd.read_csv("y_test.csv")
print(roc_auc_score(y_test, pred_proba[:,1]))

0.8776223776223776
